# Legal Policy Explainer - Demo Notebook

This notebook demonstrates the key features of the Legal Policy Explainer system.

## Setup

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv()

print("Setup complete!")

## 1. Prompt Design Demo

Demonstrates the carefully crafted system prompts for different roles.

In [ ]:
from src.prompts import get_system_prompt, add_disclaimer

# Show explainer prompt
print("=" * 80)
print("EXPLAINER AGENT SYSTEM PROMPT")
print("=" * 80)
print(get_system_prompt(role="explainer", include_examples=False)[:500] + "...")

# Show researcher prompt
print("\n" + "=" * 80)
print("RESEARCHER AGENT SYSTEM PROMPT")
print("=" * 80)
print(get_system_prompt(role="researcher", include_examples=False)[:500] + "...")

## 2. Document Processing & RAG Demo

Shows how documents are processed and retrieved.

In [ ]:
from src.rag import DocumentProcessor, VectorStore, RAGRetriever

# Process documents
print("Processing documents...")
processor = DocumentProcessor()
documents = processor.load_documents()
print(f"Loaded {len(documents)} document chunks")

# Show sample document
if documents:
    print("\nSample document chunk:")
    print(f"Source: {documents[0].metadata.get('source')}")
    print(f"Content: {documents[0].content[:300]}...")

In [ ]:
# Create vector store
print("Initializing vector store...")
vector_store = VectorStore()

# Add documents if not already added
if vector_store.get_stats()["document_count"] == 0:
    print("Adding documents to vector store...")
    vector_store.add_documents(documents)

print(f"\nVector store stats:")
print(vector_store.get_stats())

In [ ]:
# Test retrieval
query = "What is a non-disclosure agreement?"
print(f"Searching for: {query}")

results = vector_store.search(query, top_k=3)

print(f"\nFound {len(results)} relevant documents:")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Source: {result['metadata'].get('source')}")
    print(f"   Content: {result['content'][:200]}...")

## 3. Tool/Function Calling Demo

Demonstrates the legal tools available for function calling.

In [ ]:
from src.tools import LegalTools

# Initialize tools
tools = LegalTools(vector_store=vector_store)

# Show available tools
print("Available tools:")
for i, tool_def in enumerate(tools.get_tool_definitions(), 1):
    print(f"{i}. {tool_def['function']['name']}")
    print(f"   Description: {tool_def['function']['description']}")
    print()

In [ ]:
# Test legal definition tool
print("Testing get_legal_definition tool:")
result = tools.get_legal_definition("contract")
print(result)

In [ ]:
# Test document search tool
print("Testing search_legal_documents tool:")
result = tools.search_legal_documents("copyright protection", top_k=2)
print(result)

## 4. Multi-Agent System Demo

Shows how Researcher and Explainer agents collaborate.

In [ ]:
from src.llm_client import LLMClient
from src.agents import MultiAgentOrchestrator

# Initialize LLM client
print("Initializing LLM client...")
llm_client = LLMClient()

# Initialize retriever
retriever = RAGRetriever(vector_store=vector_store)

# Initialize multi-agent orchestrator
print("Initializing multi-agent system...")
orchestrator = MultiAgentOrchestrator(
    llm_client=llm_client,
    rag_retriever=retriever,
    tools=tools
)

print("Multi-agent system ready!")

In [ ]:
# Test query with multi-agent system
query = "Explain what intellectual property is and its main types."
print(f"Query: {query}\n")
print("Processing with multi-agent system...\n")

response = orchestrator.process_query(query)

print("=" * 80)
print("EXPLANATION:")
print("=" * 80)
print(response["explanation"])

print("\n" + "=" * 80)
print(f"Retrieved {len(response['retrieved_documents'])} documents")
print(f"Agents used: {', '.join(response['agents_used'])}")

## 5. Safety & Ethics Demo

Demonstrates the safety filter and disclaimer system.

In [ ]:
from src.agents import SafetyFilter

safety_filter = SafetyFilter()

# Test various queries
test_queries = [
    "What is a contract?",
    "How can I avoid paying taxes?",
    "What should I do in my lawsuit?",
    "Will I win my case?"
]

for query in test_queries:
    result = safety_filter.check_query(query)
    print(f"Query: {query}")
    print(f"Allowed: {result['allowed']}")
    if 'message' in result:
        print(f"Message: {result['message'][:100]}...")
    print()

## 6. Evaluation Demo

Shows how the system is evaluated.

In [ ]:
from src.evaluation import LegalExplainerEvaluator

# Create evaluator
evaluator = LegalExplainerEvaluator()

# Test with orchestrator
def system_func(query):
    return orchestrator.get_explanation(query)

# Note: This will make API calls and may take time
print("Running evaluation...")
print("(This may take a few minutes)\n")

results = evaluator.evaluate_system(system_func)
evaluator.print_summary(results)
evaluator.save_results(results)

## 7. Complete System Demo

Use the full application.

In [ ]:
from app import LegalPolicyExplainer

# Initialize complete system
assistant = LegalPolicyExplainer(
    use_rag=True,
    use_multi_agent=True,
    use_tools=True
)

# Test queries
test_queries = [
    "What is fair use in copyright law?",
    "Explain the difference between a patent and a trademark.",
    "What are the requirements for a valid contract?"
]

for query in test_queries:
    print("\n" + "=" * 80)
    print(f"YOU: {query}")
    print("=" * 80)
    response = assistant.query(query)
    print(f"ASSISTANT: {response}")

## Conclusion

This notebook demonstrated all key components of the Legal Policy Explainer:

1. **Prompt Design**: Specialized system prompts for different roles
2. **RAG**: Document processing and semantic retrieval
3. **Tools**: Function calling for enhanced functionality
4. **Multi-Agent**: Collaboration between Researcher and Explainer
5. **Safety**: Content filtering and disclaimers
6. **Evaluation**: Systematic assessment of system performance

The system successfully integrates all required LLM technologies for the course project.